In [1]:
from bs4 import BeautifulSoup
import urllib.request
from datetime import date
import pandas as pd
date_today = date.today()

In [145]:
urls = ['https://fearofgod.com/collections/nike-air-fear-of-god', 
        'https://fearofgod.com/collections/fear-of-god-zegna',
        'https://fearofgod.com/collections/essentials']

brand = []
name = []
timestamp = []
price = []
sales_price = []
product_type = []
product_url = []
picture_url = []

for link in urls:
    
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    
    for i in soup.find_all('div',{'class':'grid-item is-loading'}):
        timestamp.append(date_today)
        brand.append("Fear of God - " + i.find('div',{'class':'product-title'}).h4.text)
        name.append(i.find('div',{'class':'product-title'}).h3.text)
        sales_price.append(i.find('div',{'class':'title-price grey type-a'}).span.text.strip())
        price.append(None)
        product_type.append(None)
        product_url.append("https://fearofgod.com" + i.a.get('href'))
        picture_url.append("https:" + i.img.get('data-src'))
    
    for i in soup.find_all('div',{'class':'grid-item is-loading sold-out'}):
        timestamp.append(date_today)
        brand.append("Fear of God - " + i.find('div',{'class':'product-title'}).h4.text)
        name.append(i.find('div',{'class':'product-title'}).h3.text)
        sales_price.append(i.find('div',{'class':'title-price grey type-a'}).span.text.strip())
        price.append(None)
        product_type.append(None)
        product_url.append("https://fearofgod.com" + i.a.get('href'))
        picture_url.append("https:" + i.img.get('data-src'))

# Second Layer

In [147]:
product_description = []
material = []
picture_name = []
counter = 1
color = []

for url in product_url:
    item_material = []
    ourUrl = urllib.request.urlopen(url)
    soup = BeautifulSoup(ourUrl,'html.parser')
    product_description.append(soup.find('div',{'class':'product-description type-f'}).text.strip().replace('\xa0',' ').replace('\n',' '))

    if soup.find('div',{'class':'text type-i'}) != None:
        for material_i in soup.find('div',{'class':'text type-i'}).ul.find_all('li'):
            if material_i.text != None:
                item_material.append(material_i.text)
        material.append(item_material)
    else:
        material.append(None)
        
    color.append(None)
    picture_name.append('fear-of-god_' + str(counter) + '.jpg') # change variable
    counter += 1

In [149]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Fear of God - Air Fear of God,None,Short Sleeve Warm Up Shirt,"Inspired by 90's NBA, the warm up shirt delive...",None,$ 150,None,"[100% nylon, 100% nylon mesh lining, NBA logo ...",https://fearofgod.com/collections/nike-air-fea...,https://cdn.shopify.com/s/files/1/1003/3354/pr...,2020-12-06,fear-of-god_1.jpg
1,Fear of God - Air Fear of God,None,Short Sleeve Shooting Shirt,"Inspired by 90's NBA, the short sleeve shootin...",None,$ 150,None,"[100% nylon, 100% nylon mesh lining, NBA logo ...",https://fearofgod.com/collections/nike-air-fea...,https://cdn.shopify.com/s/files/1/1003/3354/pr...,2020-12-06,fear-of-god_2.jpg
2,Fear of God - Air Fear of God,None,Basketball Warm Up Jacket,"Inspired by 90's NBA, the basketball warm up j...",None,$ 250,None,"[100% nylon, 100% nylon mesh lining, Jetted Po...",https://fearofgod.com/collections/nike-air-fea...,https://cdn.shopify.com/s/files/1/1003/3354/pr...,2020-12-06,fear-of-god_3.jpg
3,Fear of God - Air Fear of God,None,Air Fear of God Tee Shirt,"Inspired by 90's NBA, the basketball tee shirt...",None,$ 45,None,"[100% cotton, NBA logo on left chest, Nike Swo...",https://fearofgod.com/collections/nike-air-fea...,https://cdn.shopify.com/s/files/1/1003/3354/pr...,2020-12-06,fear-of-god_4.jpg
4,Fear of God - Air Fear of God,None,Air Fear of God Tee Shirt,"Inspired by 90's NBA, the basketball tee shirt...",None,$ 45,None,"[100% cotton, NBA logo on left chest, Nike Swo...",https://fearofgod.com/collections/nike-air-fea...,https://cdn.shopify.com/s/files/1/1003/3354/pr...,2020-12-06,fear-of-god_5.jpg


In [150]:
second_pass.to_csv("fear-of-god.csv", index=False, encoding = 'utf-8-sig')

# Download Images

In [151]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/fear-of-god.csv', encoding='utf-8-sig')

os.mkdir('fear-of-god')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('fear-of-god/'+ "fear-of-god_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1